In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-darkgrid')

import requests
from time import sleep
from bs4 import BeautifulSoup
import csv
from urllib.parse import urlparse
from random import shuffle, sample

In [2]:
bing_df = pd.read_csv('data/bing_search_results.csv')
domains = set()
with open('data/scraped_urls.txt', 'r') as file:
    urls = set([url.strip() for url in file])
len(urls)

3513

In [3]:
def random_search_result():
    return bing_df.sample(1).iloc[0].url

In [ ]:
url = None
depth = 0
search_count = 0
request_count = 0
i = 0
while len(urls) < 1e4:
    sleep(0.5)
    if not url:
        url = random_search_result()
        search_count += 1
        while urlparse(url).netloc in domains:
            search_count += 1
            url = random_search_result()
        depth = 0
    try:
        request_count += 1
        response = requests.get(url)
    except:
        url = None
        continue
    soup = BeautifulSoup(response.text,"lxml")
    a_tags = soup.find_all('a')
    shuffle(a_tags)
    url = None
    for a_tag in a_tags:
        href = a_tag.get('href')
        if href and href[0:4] == 'http':
            domain = urlparse(href).netloc
            if not domain in domains:
                domains.add(domain)
                urls.add(href)
                url = href
    depth += 1
    i += 1
    if depth > 100: url = None
    with open('data/scraped_urls.txt', 'w') as file:
        file.write("\n".join(urls))
    print('depth: {}, urls: {}, search: {}, requests: {}, iters: {} '.format(depth, len(urls), search_count, request_count, i), end='\r')
    

In [ ]:
with open('data/scraped_urls.txt', 'w') as file:
    file.write("\n".join(urls))